Notebook to evaluate BraTS18 image data based on segmentation label ratios

Goal is to filter out only samples above a cerain theshold.

In [1]:
import numpy as np
import os

In [2]:
def crop_with_segmentation(image):
    # Assuming the input image shape is [5, 155, 240, 240]
    # The last channel is the segmentation mask, shape [155, 240, 240]
    seg_mask = image[-1, :, :, :]  # Extract the segmentation mask (last channel)
    
    # Find the indices where the segmentation mask is non-zero
    non_zero_indices = np.nonzero(seg_mask)
    
    # Get the bounding box of the non-zero regions
    min_z, min_y, min_x = np.min(non_zero_indices, axis=1)
    max_z, max_y, max_x = np.max(non_zero_indices, axis=1)
    
    # Calculate the center of the bounding box
    center_z = (min_z + max_z) // 2
    center_y = (min_y + max_y) // 2
    center_x = (min_x + max_x) // 2
    
    # Define the cropping size (half the size of the original in each direction)
    crop_z = 80
    crop_y = 240 // 2
    crop_x = 240 // 2
    
    # Define the starting and ending indices for cropping, ensuring that the crop is within bounds
    start_z = max(center_z - crop_z // 2, 0)
    end_z = min(center_z + crop_z // 2, 155)
    
    start_y = max(center_y - crop_y // 2, 0)
    end_y = min(center_y + crop_y // 2, 240)
    
    start_x = max(center_x - crop_x // 2, 0)
    end_x = min(center_x + crop_x // 2, 240)
    
    # Crop the image to the new dimensions, ensuring the segmentation mask remains within the crop
    cropped_image = image[:, start_z:end_z, start_y:end_y, start_x:end_x]
    
    return cropped_image

In [3]:
# Folder path for files
folder_path = 'npy/'
files = os.listdir(folder_path)
npy_files = [f for f in files if f.lower().endswith('.npy')]

# Load the first .npy file
image_first = np.load(os.path.join(folder_path, npy_files[1]))

# Get the unique values and their counts from the mask (assuming it's the last channel: index 4)
unique_values, counts = np.unique(image_first[4], return_counts=True)
print(counts[1:])

# Sum the counts (second element of the tuple)
sum_counts = np.sum(counts[1:])

print("shape before crop: ", image_first.shape)
print("sum mask before crop: ", sum_counts)

# Assuming crop_with_segmentation is defined elsewhere
image_cropped = crop_with_segmentation(image_first)

# Get unique values and counts after cropping
unique_values_cropped, counts_cropped = np.unique(image_cropped[4], return_counts=True)
sum_counts_cropped = np.sum(counts_cropped[1:])

print("shape after crop: ", image_cropped.shape)
print("sum mask after crop: ", sum_counts_cropped)
print("percentage of seg pixels: ", (sum_counts_cropped/sum_counts*100))

[27189 53050 31485]
shape before crop:  (5, 155, 240, 240)
sum mask before crop:  111724
shape after crop:  (5, 80, 120, 120)
sum mask after crop:  111724
percentage of seg pixels:  100.0


In [4]:
import os
import numpy as np

# Folder path for the input files
folder_path = 'npy/'
# Create a new folder for cropped images if it doesn't exist
cropped_folder_path = 'npy_cropped/'
if not os.path.exists(cropped_folder_path):
    os.makedirs(cropped_folder_path)

# List all .npy files in the folder
files = os.listdir(folder_path)
npy_files = [f for f in files if f.lower().endswith('.npy')]

# Function to process and crop images
def process_and_crop_images():
    for npy_file in npy_files:
        # Load the .npy file
        image = np.load(os.path.join(folder_path, npy_file))
        
        # Get the unique values and their counts from the mask (assuming it's the last channel: index 4)
        unique_values, counts = np.unique(image[4], return_counts=True)
        
        # Sum the counts (excluding the background, assuming it's label 0)
        sum_counts = np.sum(counts[1:])
        
        # Perform cropping (assuming crop_with_segmentation is defined elsewhere)
        image_cropped = crop_with_segmentation(image)
        
        # Get unique values and counts after cropping
        unique_values_cropped, counts_cropped = np.unique(image_cropped[4], return_counts=True)
        sum_counts_cropped = np.sum(counts_cropped[1:])
        
        # Calculate the percentage of segmented pixels in the cropped image
        percentage_of_seg_pixels = (sum_counts_cropped / sum_counts) * 100
        print(f"File: {npy_file} - Percentage of segmentation pixels: {percentage_of_seg_pixels:.2f}%")
        
        # Save the cropped image to the new folder
        save_path = os.path.join(cropped_folder_path, npy_file)
        np.save(save_path, image_cropped)
        print(f"Cropped image saved to: {save_path}")

# Call the function to process all images
process_and_crop_images()


File: Brats18_2013_0_1.npy - Percentage of segmentation pixels: 100.00%
Cropped image saved to: npy_cropped/Brats18_2013_0_1.npy
File: Brats18_2013_10_1.npy - Percentage of segmentation pixels: 100.00%
Cropped image saved to: npy_cropped/Brats18_2013_10_1.npy
File: Brats18_2013_11_1.npy - Percentage of segmentation pixels: 100.00%
Cropped image saved to: npy_cropped/Brats18_2013_11_1.npy
File: Brats18_2013_12_1.npy - Percentage of segmentation pixels: 100.00%
Cropped image saved to: npy_cropped/Brats18_2013_12_1.npy
File: Brats18_2013_14_1.npy - Percentage of segmentation pixels: 100.00%
Cropped image saved to: npy_cropped/Brats18_2013_14_1.npy
File: Brats18_2013_17_1.npy - Percentage of segmentation pixels: 99.99%
Cropped image saved to: npy_cropped/Brats18_2013_17_1.npy
File: Brats18_2013_19_1.npy - Percentage of segmentation pixels: 100.00%
Cropped image saved to: npy_cropped/Brats18_2013_19_1.npy
File: Brats18_2013_20_1.npy - Percentage of segmentation pixels: 99.91%
Cropped image 